In [1]:
import numpy as np

In [2]:
def elem_stiffness(E, A, n1, n2):
    #coordinates from nodes
    x1, y1 = n1
    x2, y2 = n2
    #distance calc
    L = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    #local stiffness matrix
    k_local = (E * A / L) * np.array([[1, 0, -1, 0],
                                      [0,0,0,0],
                                       [-1, 0, 1, 0],
                                         [0,0,0,0]])
    #Global Stiffness
    #direction trig
    C = (x2 - x1) / L
    S = (y2 - y1) / L
    #transformation matrix
    b = np.array([[C, S, 0, 0],
                  [-S, C, 0, 0],
                  [0, 0, C, S],
                  [0, 0, -S, C]])
    #calc global stiffness
    k_global = b.T @ k_local @ b
    
    return k_local, k_global, L, C, S

    

In [3]:
#displacement field
#Givens
E = 29e6  # psi
A = 3.6   # in²
n1 = (0, 0)
n2 = (-100, -100)
n3 = (100, -100)
elements = [(n1, n2), (n1, n3), (n2, n3)]
F = np.array([100000, 0, 0, 0, 0, 0])

# Global stiffness matrix (3 nodes, each with 2 dof)
K_global = np.zeros((6, 6))

# Assemble Global Stiffness Matrix
lengths = []
cosines = []
sines = []
for n, (ni, nj) in enumerate(elements):
    k_local, k_global, L, C, S = elem_stiffness(E, A, ni, nj)
    lengths.append(L)
    cosines.append(C)
    sines.append(S)
    
    dof_map = [2*n, 2*n+1, 2*n+2, 2*n+3]
    
    for i in range(4):
        for j in range(4):
            K_global[dof_map[i] % 6, dof_map[j] % 6] += k_global[i, j]

# Boundry conditions on nodes 2 and 3 
free_dofs = [0, 1]  # Only n1's displacements are unknown
K_reduced = K_global[np.ix_(free_dofs, free_dofs)]
F_reduced = F[free_dofs]

# Solve for displacements
U = np.zeros(6)
U[free_dofs] = np.linalg.solve(K_reduced, F_reduced)

print("Displacement field:")
print(U)


Displacement field:
[ 0.19157088 -0.19157088  0.          0.          0.          0.        ]


In [4]:
# Compute reaction forces
reaction_forces = K_global @ U - F
fixed_dofs = [2, 3, 4, 5]

print("Reaction forces at fixed nodes:")
print(reaction_forces[fixed_dofs])

Reaction forces at fixed nodes:
[      0.       0. -100000.       0.]


In [5]:
#internal forces
internal_forces = []
nodes = [n1, n2, n3]
for i, (ni, nj) in enumerate(elements):
    L = lengths[i]
    C = cosines[i]
    S = sines[i]
    
    node_i = nodes.index(ni)
    node_j = nodes.index(nj)
    
    dof_map = [2*node_i, 2*node_i+1, 2*node_j, 2*node_j+1]  # Proper DOF mapping
    
    # Global displacements for the element
    u_global = U[dof_map]
    
    # Transformation matrix
    T = np.array([[C, S, 0, 0],
                  [0, 0, C, S]])
    
    # Global displacements to local displacements
    u_local = T @ u_global
    
    # axial force equation
    force = (E * A / L) * (u_local[1] - u_local[0])
    internal_forces.append(force)

print("Internal forces in each element:")
print(internal_forces)

Internal forces in each element:
[0.0, -199999.99999999997, 0.0]
